In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/.shortcut-targets-by-id/1bqqm9-0FMOqiqyH_NIQBh-lJ36pWa7aX/Masters_Project/Datasets/OASIS3


In [63]:
!ls

 cifar-10-batches-py	  fa20		    t1w_non_ad_patients
 cifar-10-python.tar.gz   oasis-scripts    'Usage Instructions.gdoc'
 data			  t1w_ad_jpgs
 downloaded_data	  t1w_non_ad_jpgs


In [4]:
!pip install -r ../../Code/ganomaly/requirements.txt
#!pip install -r ./ganomaly/requirements.txt

  Using cached https://files.pythonhosted.org/packages/fc/d1/45be1144b03b6b1e24f9a924f23f66b4ad030d834ad31fb9e5581bd328af/numpy-1.16.4-cp37-cp37m-manylinux1_x86_64.whl
ERROR: umap-learn 0.5.1 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mkl-fft 1.2.0 has requirement numpy==1.19.5, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have mat

In [5]:
!pip install mkl-fft

  Using cached https://files.pythonhosted.org/packages/08/d6/a6aaa29fea945bc6c61d11f6e0697b325ff7446de5ffd62c2fa02f627048/numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: librosa 0.8.0 has requirement joblib>=0.14, but you'll have joblib 0.13.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.15.0, bu

In [6]:
!pip install Pillow

# imports

In [7]:
import os
#import os.path
#import cv2
import glob
import torch
import random
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from PIL import Image

from sklearn.model_selection import train_test_split

import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [8]:
print(torch.__version__)

1.2.0


In [9]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [10]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])# https://github.com/yunjey/pytorch-tutorial/issues/161
        
    ]
)

In [52]:
train_image_number = 140 # Image slice from MRI scans to use for training data
root_dir = '/content/drive/MyDrive/Masters_Project/Datasets/OASIS3/'
base_folder = '/pyt_oasis3'
non_ad_jpg_folder = './t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = './t1w_ad_jpgs/' + str(train_image_number) + '/'

class Oasis3(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set

        # Set repeatble random number
        
        self.non_ad_data = []
        self.non_ad_targets = []

        self.ad_data = []
        self.ad_targets = []
        
        # load both datasets
        for file_path in glob.glob(ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.ad_data.append(resized)
                self.ad_targets.append(1) # based on the jpg_folder in for loop
        self.ad_data = np.vstack(self.ad_data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.ad_data = self.ad_data.transpose((0, 1, 2))  # convert to HWC

        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.non_ad_data.append(resized)
                self.non_ad_targets.append(0) # based on the jpg_folder in for loop
        self.non_ad_data = np.vstack(self.non_ad_data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.non_ad_data = self.non_ad_data.transpose((0, 1, 2))  # convert to HWC
        
        # shuffle the datasets and bucket them as test and train
        (self.non_ad_data, self.non_ad_targets) = self.shuffle_associated_arrays(self.non_ad_data, self.non_ad_targets)
        (self.ad_data, self.ad_targets) = self.shuffle_associated_arrays(self.ad_data, self.ad_targets)

        all_data = self.non_ad_data + self.ad_data
        all_targets = self.non_ad_targets + self.ad_targets
        # concat the lists and random shuffly
        random.Random(1).shuffle(all_data)
        random.Random(1).shuffle(all_targets)

        # Use scikit learn to split complete data into test and train
        self.train_data, self.test_data, self.train_targets, self.test_targets =\
         train_test_split(all_data, all_targets, test_size=0.1, random_state=42)

        # Set test or train data based on input
        if self.train:
          self.data = self.train_data
          self.targets = self.train_targets
        else:
          self.data = self.test_data
          self.targets = self.test_targets

    def shuffle_associated_arrays(self, arr1, arr2):
        c = list(zip(arr1, arr2))
        random.shuffle(c)
        arr1, arr2 = zip(*c)
        return (list(arr1), list(arr2))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [53]:
# Issue is i need to make the data transformable inside __getitem__
a=Oasis3_train_normal(base_folder, False, transform=transform)

dataset = {}
dataset['train'] = Oasis3(root='./data', train=True, transform=transform)
dataset['test'] = Oasis3(root='./data', train=False, transform=transform)

In [54]:
# The class should be index into which invokes the __getitem__ call
print(dataset['train'][0][0].shape)
print(dataset['train'][0][0])
print(dataset['train'][0][1])

torch.Size([1, 256, 256])
tensor([[[-0.9686, -0.9529, -0.9451,  ..., -0.9843, -0.9529, -1.0000],
         [-0.9686, -0.9608, -0.9686,  ..., -0.9529, -0.9529, -1.0000],
         [-0.9765, -0.9686, -0.9922,  ..., -0.9451, -0.9608, -1.0000],
         ...,
         [-0.9922, -0.9294, -0.9922,  ..., -0.9686, -0.9843, -0.9922],
         [-0.9843, -0.9451, -0.9922,  ..., -0.9765, -0.9922, -0.9922],
         [-0.9843, -0.9765, -0.9686,  ..., -0.9765, -0.9922, -1.0000]]])
0


In [61]:
def get_oasis_anomaly_dataset(trn_img, trn_lbl, tst_img, tst_lbl, abn_cls_idx=1, manualseed=-1):
    """[summary]
    Arguments:
        trn_img {np.array} -- Training images
        trn_lbl {np.array} -- Training labels
        tst_img {np.array} -- Test     images
        tst_lbl {np.array} -- Test     labels
    Keyword Arguments:
        abn_cls_idx {int} -- Anomalous class index (default: {1})
    Returns:
        [np.array] -- New training-test images and labels.
    """
    # Convert train-test labels into numpy array.
    trn_lbl = np.array(trn_lbl)
    tst_lbl = np.array(tst_lbl)

    # --
    # Find idx, img, lbl for abnormal and normal on org dataset.
    nrm_trn_idx = list(np.where(trn_lbl != abn_cls_idx))[0]
    abn_trn_idx = list(np.where(trn_lbl == abn_cls_idx))[0]

    print(nrm_trn_idx)

    # nrm_trn_img = trn_img[nrm_trn_idx]    # Normal training images
    # abn_trn_img = trn_img[abn_trn_idx]    # Abnormal training images
    # nrm_trn_lbl = trn_lbl[nrm_trn_idx]    # Normal training labels
    # abn_trn_lbl = trn_lbl[abn_trn_idx]    # Abnormal training labels.

    nrm_trn_img = [trn_img[i] for i in nrm_trn_idx]
    abn_trn_img = [trn_img[i] for i in abn_trn_idx]
    nrm_trn_lbl = [trn_lbl[i] for i in nrm_trn_idx]
    abn_trn_img = [trn_lbl[i] for i in abn_trn_idx]  

    nrm_tst_idx = list(np.where(tst_lbl != abn_cls_idx))[0]
    abn_tst_idx = list(np.where(tst_lbl == abn_cls_idx))[0]

    # nrm_tst_img = tst_img[nrm_tst_idx]    # Normal training images
    # abn_tst_img = tst_img[abn_tst_idx]    # Abnormal training images.
    # nrm_tst_lbl = tst_lbl[nrm_tst_idx]    # Normal training labels
    # abn_tst_lbl = tst_lbl[abn_tst_idx]    # Abnormal training labels.

    nrm_tst_img = [tst_img[i] for i in nrm_tst_idx]
    abn_tst_img = [tst_img[i] for i in abn_tst_idx]
    nrm_tst_lbl = [trn_lbl[i] for i in nrm_tst_idx]
    abn_tst_lbl = [trn_lbl[i] for i in abn_tst_idx]  

    # --
    # Assign labels to normal (0) and abnormals (1)
    nrm_trn_lbl[:] = 0
    nrm_tst_lbl[:] = 0
    abn_trn_lbl[:] = 1
    abn_tst_lbl[:] = 1

    # --
    if manualseed != -1:
        # Random seed.
        # Concatenate the original train and test sets.
        nrm_img = np.concatenate((nrm_trn_img, nrm_tst_img), axis=0)
        nrm_lbl = np.concatenate((nrm_trn_lbl, nrm_tst_lbl), axis=0)
        abn_img = np.concatenate((abn_trn_img, abn_tst_img), axis=0)
        abn_lbl = np.concatenate((abn_trn_lbl, abn_tst_lbl), axis=0)

        # Split the normal data into the new train and tests.
        idx = np.arange(len(nrm_lbl))
        np.random.seed(manualseed)
        np.random.shuffle(idx)

        nrm_trn_len = int(len(idx) * 0.80)
        nrm_trn_idx = idx[:nrm_trn_len]
        nrm_tst_idx = idx[nrm_trn_len:]

        nrm_trn_img = nrm_img[nrm_trn_idx]
        nrm_trn_lbl = nrm_lbl[nrm_trn_idx]
        nrm_tst_img = nrm_img[nrm_tst_idx]
        nrm_tst_lbl = nrm_lbl[nrm_tst_idx]

    # Create new anomaly dataset based on the following data structure:
    # - anomaly dataset
    #   . -> train
    #        . -> normal
    #   . -> test
    #        . -> normal
    #        . -> abnormal
    new_trn_img = np.copy(nrm_trn_img)
    new_trn_lbl = np.copy(nrm_trn_lbl)
    new_tst_img = np.concatenate((nrm_tst_img, abn_trn_img, abn_tst_img), axis=0)
    new_tst_lbl = np.concatenate((nrm_tst_lbl, abn_trn_lbl, abn_tst_lbl), axis=0)

    return new_trn_img, new_trn_lbl, new_tst_img, new_tst_lbl

In [62]:
dataset['train'].data, dataset['train'].targets, \
dataset['test'].data, dataset['test'].targets = get_oasis_anomaly_dataset(
    trn_img=dataset['train'].data,
    trn_lbl=dataset['train'].targets,
    tst_img=dataset['test'].data,
    tst_lbl=dataset['test'].targets,
    abn_cls_idx=1
)

[   0    2    3    4    5    8   10   11   14   16   17   18   19   20
   21   22   23   24   25   26   28   29   31   33   35   36   38   40
   41   42   43   44   45   46   48   49   51   52   53   55   56   57
   58   59   60   61   62   63   64   66   67   68   69   70   71   72
   73   74   75   76   77   78   79   82   83   85   86   87   88   89
   90   93   94   95   97   98  101  102  103  105  108  109  110  111
  112  113  114  115  116  117  118  119  122  123  125  127  128  130
  133  135  140  141  142  143  144  145  146  147  148  149  150  151
  153  155  156  157  158  160  161  162  164  165  167  168  169  170
  171  172  173  174  175  177  178  179  180  181  184  186  187  189
  192  193  194  195  196  197  198  199  200  202  203  204  206  208
  209  211  212  213  214  215  218  220  221  222  223  224  225  226
  227  229  230  232  233  234  235  236  238  240  242  243  244  245
  246  247  248  250  252  253  255  256  257  258  259  260  261  262
  263 

TypeError: ignored

In [40]:
train_label = dataset['train'].targets

In [44]:
train_label = np.array(train_label)

In [46]:
trn_lbl_no_ad = np.where(train_label != 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.
